In [21]:
import os, csv, pickle, json, pymongo, copy, re, time, datetime
import pandas as pd
from pymongo import MongoClient

In [22]:
pd.set_option('display.max_columns', 50)

In [23]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [24]:
len(jcr)

21430

In [25]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
21420,12425,AGENDA,AGENDA,2022,NaN,0002-0796,159,0.0,0.0,0.0,0.0,0,17.8,0.0,0.00005,0.114,NaN,NaN,NaN,NaN,NaN,0.0,0.01206
21421,12425,Abakos,ABAKOS,2022,NaN,2316-9451,7,0.0,0.0,0.0,0.0,18,0.0,10.7,0.00003,0.068,NaN,NaN,NaN,NaN,NaN,0.0,0.00733
21422,12425,Adalya,ADALYA,2022,NaN,1301-2746,75,0.0,0.0,0.0,0.0,19,0.0,16.2,0.00027,0.306,NaN,NaN,NaN,NaN,NaN,0.0,0.05846
21423,12425,Anduli,ANDULI,2022,NaN,1696-0270,35,0.0,0.0,0.0,0.0,15,0.0,14.6,0.00007,0.150,NaN,NaN,NaN,NaN,NaN,0.0,0.01555
21424,12425,Anemia,ANEMIA,2022,NaN,2090-1267,502,0.0,0.0,0.0,0.0,14,7.3,7.7,0.00030,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.06589
21425,12425,Appeal,APPEAL,2022,NaN,1205-612X,56,0.0,0.0,0.0,0.0,6,0.0,4.8,0.00019,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.04134
21426,12425,Atenas,ATENAS,2022,NaN,1682-2749,60,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00007,0.034,NaN,NaN,NaN,NaN,NaN,0.0,0.01561
21427,12425,Atenea,ATENEA-CONCEPC,2022,NaN,0718-0462,105,0.0,0.0,0.0,0.0,35,8.8,15.8,0.00034,0.281,NaN,NaN,NaN,NaN,NaN,0.0,0.07463
21428,12425,AusArt,AUSART,2022,NaN,2340-9134,13,0.0,0.0,0.0,0.0,32,0.0,11.8,0.00002,0.017,NaN,NaN,NaN,NaN,NaN,0.0,0.00594


In [26]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [27]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [28]:
list(jt2if_brief.items())[:2]

[('ca-cancer j clin', 286.13), ('lancet', 202.731)]

In [29]:
len(jt2if_brief)

21429

In [30]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [31]:
jt2if5_brief={}
for i in range(len(jcr)):
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [32]:
list(jt2if5_brief.items())[:2]

[('ca-cancer j clin', 334.259), ('lancet', 130.838)]

In [33]:
client = MongoClient(username="hald",password="mclab236")

In [34]:
db = client.hald

In [35]:
collections_pubtator_info = db.pubtator_info

In [36]:
pubtator_info=list(collections_pubtator_info.find())

In [37]:
len(pubtator_info)

206395

In [38]:
pubtator_info[10]

{'_id': ObjectId('6393478a5303aaecac2745e7'),
 'PMID': '35794516',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20220708',
 'LR': '20220708',
 'IS': '1471-227X (Electronic) 1471-227X (Linking)',
 'VI': '22',
 'IP': '1',
 'DP': '2022 Jul 6',
 'TI': 'Long-term prognosis and clinical course of choking-induced cardiac arrest in patients without the return of spontaneous circulation at hospital arrival: a population-based community study from the Shizuoka Kokuho Database.',
 'PG': '120',
 'LID': '10.1186/s12873-022-00676-8 [doi]',
 'AB': 'BACKGROUND: The risk of choking increases with aging, and the number of cases of choking-induced cardiac arrest is increasing. However, few studies have examined the prognosis of choking-induced cardiac arrest. The aim of this study was to reveal the rates of survival and dependence on devices in the long term after choking-induced cardiac arrest. METHODS: We analyzed data from the Shizuoka Kokuho Database, which consists of claims data of approximately 2.

In [39]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [40]:
collections_Relations_final=db.Relations_final

In [41]:
triplelist=list(collections_Relations_final.find())

In [42]:
triplelist[0]

{'_id': ObjectId('64611763a2e9cde11765b7be'),
 'PMID': '30781849',
 'entity1': 'Pulmonary Disease, Chronic Obstructive',
 'entity2': 'Inflammation',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tob

In [43]:
len(triplelist)

149274

In [44]:
triplelist[0]

{'_id': ObjectId('64611763a2e9cde11765b7be'),
 'PMID': '30781849',
 'entity1': 'Pulmonary Disease, Chronic Obstructive',
 'entity2': 'Inflammation',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tob

In [45]:
be=['am','is','are','was','were','been','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'

triplelist_takeoff=[]
transformed_triples=set()
box=[]
box2=[]
acount=0
for i in triplelist:
    maxlength=[]
    allennlp_list=[]
    if i.get('openie_final'):
        maxlength=i.get('openie_final')[0]
        for j in i.get('openie_final'):
            if len(j[1])>len(maxlength[1]):
                maxlength=j
        mid_list=maxlength[1].split(' ')
        for p,q in enumerate(mid_list):
            if bedict.get(q):
                mid_list[p]='be'
        maxlength[1]=' '.join(mid_list)
        for k in maxlength[1]:
            if k in maxlength[0].split(' ') or k in maxlength[2].split(' '):
                break
        else:
            mid=tuple([maxlength[0],maxlength[1].lower(),maxlength[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if maxlength[1]=='associated' or maxlength[1]=='associate' :
                mid=tuple([maxlength[2],maxlength[1].lower(),maxlength[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
        for j in i.get('openie_final'):
            if j==maxlength:
                continue
            judge=True
            for k in j[1].split(' '):
                if k not in maxlength[1].split(' '):
                    judge=False
                    break
            if not judge:
                mid_list=j[1].split(' ')
                for p,q in enumerate(mid_list):
                    if bedict.get(q):
                        mid_list[p]='be'
                j[1]=' '.join(mid_list)
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                for k in j[1]:
                    if k in j[0].split(" ") or k in j[1].split(" "):
                        break
                else:
                    if mid not in transformed_triples:
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                                  'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                                  'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    if j[1]=='associated' or j[1]=='associate' :
                        mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                        if mid not in transformed_triples:
                            copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                      'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                      'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                      'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                        'transformed_triple':mid}
                            triplelist_takeoff.append(copydict)
                            transformed_triples.add(mid)
    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            if len(maxlength)>0 and j[1] in maxlength[1].split(' '):
                pass
            else:
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                if j[1] in j[0].split(' ') or j[1] in j[2].split(' '):
                    continue
                if mid not in transformed_triples:
                    allennlp_list.append((j[0],j[2]))
                    copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                              'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                              'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
                else:
                    box2.append(i)
                if j[1]=='associated' or j[1]=='associate' :
                    mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                    if mid not in transformed_triples:
                        allennlp_list.append([j[2],j[0]])
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                  'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                  'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    else:
                        box2.append(i)
    if i.get('networkx'):
        if len(maxlength)>0 and i.get('networkx')[1] in maxlength[1].split(' '):
            box.append(i)
            pass
        else:
            if i.get('networkx')[1] in i.get('networkx')[0].split(' ') or i.get('networkx')[1] in i.get('networkx')[2].split(' '):
                acount+=1
                continue
            mid=tuple([i.get('networkx')[0],i.get('networkx')[1].lower(),i.get('networkx')[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
                mid=tuple([i.get('networkx')[2],i.get('networkx')[1].lower(),i.get('networkx')[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)

In [46]:
triplelist_takeoffed=[]
transformed_triples=set()
box=[]
box2=[]
acount=0
for i in triplelist:
    maxlength=[]
    allennlp_list=[]
    if i.get('openie_final'):
        maxlength=i.get('openie_final')[0]
        for j in i.get('openie_final'):
            if len(j[1])>len(maxlength[1]):
                maxlength=j
        mid_list=maxlength[1].split(' ')
        for p,q in enumerate(mid_list):
            if bedict.get(q):
                mid_list[p]='be'
        maxlength[1]=' '.join(mid_list)
        for k in maxlength[1]:
            if k in maxlength[0].split(' ') or k in maxlength[2].split(' '):
                break
        else:
            mid=tuple([maxlength[0],maxlength[1].lower(),maxlength[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid,'method':["open domain"]}
                triplelist_takeoffed.append(copydict)
                transformed_triples.add(mid)
            if maxlength[1]=='associated' or maxlength[1]=='associate' :
                mid=tuple([maxlength[2],maxlength[1].lower(),maxlength[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid,'method':["open domain"]}
                    triplelist_takeoffed.append(copydict)
                    transformed_triples.add(mid)
        for j in i.get('openie_final'):
            if j==maxlength:
                continue
            judge=True
            for k in j[1].split(' '):
                if k not in maxlength[1].split(' '):
                    judge=False
                    break
            if not judge:
                mid_list=j[1].split(' ')
                for p,q in enumerate(mid_list):
                    if bedict.get(q):
                        mid_list[p]='be'
                j[1]=' '.join(mid_list)
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                for k in j[1]:
                    if k in j[0].split(" ") or k in j[1].split(" "):
                        break
                else:
                    if mid not in transformed_triples:
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                                  'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                                  'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                    'transformed_triple':mid,'method':["open domain"]}
                        triplelist_takeoffed.append(copydict)
                        transformed_triples.add(mid)
                    if j[1]=='associated' or j[1]=='associate' :
                        mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                        if mid not in transformed_triples:
                            copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                      'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                      'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                      'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                        'transformed_triple':mid,'method':["open domain"]}
                            triplelist_takeoffed.append(copydict)
                            transformed_triples.add(mid)
    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            if len(maxlength)>0 and j[1] in maxlength[1].split(' '):
                pass
            else:
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                if j[1] in j[0].split(' ') or j[1] in j[2].split(' '):
                    continue
                allennlp_list.append((j[0],j[2]))
                copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid,'method':["deep learning"]}
                triplelist_takeoffed.append(copydict)
                transformed_triples.add(mid)
                if j[1]=='associated' or j[1]=='associate' :
                    mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                    allennlp_list.append([j[2],j[0]])
                    copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid,'method':["deep learning"]}
                    triplelist_takeoffed.append(copydict)
                    transformed_triples.add(mid)
    if i.get('networkx'):
        if len(maxlength)>0 and i.get('networkx')[1] in maxlength[1].split(' '):
            box.append(i)
            pass
        else:
            if i.get('networkx')[1] in i.get('networkx')[0].split(' ') or i.get('networkx')[1] in i.get('networkx')[2].split(' '):
                acount+=1
                continue
            mid=tuple([i.get('networkx')[0],i.get('networkx')[1].lower(),i.get('networkx')[2],i.get('PMID')])
            copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                      'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                      'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                      'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                        'transformed_triple':mid,'method':["shortest path"]}
            triplelist_takeoffed.append(copydict)
            transformed_triples.add(mid)
            if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
                mid=tuple([i.get('networkx')[2],i.get('networkx')[1].lower(),i.get('networkx')[0],i.get('PMID')])

                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                          'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                          'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                            'transformed_triple':mid,'method':["shortest path"]}
                triplelist_takeoffed.append(copydict)
                transformed_triples.add(mid)

for i in triplelist:
    if i.get('networkx') and i.get('openie_final'):
        print(i)
        break

In [47]:
len(triplelist_takeoffed)

200734

In [48]:
triplelist_takeoffed_dict={}
triplelist_takeoff=[]
for i in triplelist_takeoffed:
    if not triplelist_takeoffed_dict.get((i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))):
        triplelist_takeoffed_dict[(i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))]=i
    else:
        if i.get('method')[0] in triplelist_takeoffed_dict.get((i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))).get('method'):
            continue
        else:
            triplelist_takeoffed_dict[(i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))]['method']=triplelist_takeoffed_dict.get((i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))).get('method')+i.get('method')

In [49]:
triplelist_takeoff=list(triplelist_takeoffed_dict.values())

In [50]:
for i in triplelist_takeoff:
    if len(i.get('method'))>1:
        print(i)
        break

{'triple': ['COPD', 'defined', 'inflammatory disorder'], 'PMID': '30781849', 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.', 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of th

In [51]:
len(box)

1448

In [52]:
len(box2)

0

In [53]:
print(len(triplelist_takeoff))
print(len(transformed_triples))

174218
170410


In [54]:
triplelist_takeoff[0]

{'triple': ['Chronic obstructive pulmonary disease',
  'defined',
  'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obta

In [55]:
triplelist_takeoff_dict={}
for i in triplelist_takeoff:
    triplelist_takeoff_dict[(i.get('entity_one_transformed'),i.get('triple')[1],i.get('entity_two_transformed'),i.get('PMID'))]=triplelist_takeoff_dict.get((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')),[])+[i]

In [56]:
len(triplelist_takeoff_dict)

156647

In [57]:
triplelist_takeoffs=[]
a=[]
b=[]
c=[]
d=[]
for i,j in triplelist_takeoff_dict.items():
    if len(j)==1:
        triplelist_takeoffs.append(j[0])
    else:
        deprecated_set=set()
        for k in j:
            if k.get('entity_one').lower()==k.get('entity_one_transformed').lower() and k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                triplelist_takeoffs.append(k)
                a.append(k)
                break
        else:
            for k in j:
                if k.get('entity_one').lower()==k.get('entity_one_transformed').lower():
                    triplelist_takeoffs.append(k)
                    b.append(k)
                    break
            else:
                for k in j:
                    if k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                        c.append(k)
                        triplelist_takeoffs.append(k)
                        break
                else:
                    d.append(k)
                    triplelist_takeoffs.append(j[0])

In [58]:
names=locals()
for i in ['a','b','c','d']:
    print(len(names[i]))

0
0
0
0


In [59]:
len(triplelist_takeoffs)

156647

In [60]:
len(triplelist_takeoff)

174218

In [61]:
triplelist_takesoff=[]
for i in triplelist_takeoffs:
    if not re.search(r'\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\{|\[|\}|\]|\;|\:|\'|\"|\,|\<|\.|\>|\/|\?',i.get('triple')[1]):
        if not re.search(r'(\s\d+\s)|(\b\d+\b)',i.get('triple')[1]): 
            triplelist_takesoff.append(i)

In [62]:
triplelistnewcopy=copy.deepcopy(triplelist_takesoff)

In [63]:
deprecated_relations=pd.read_csv('../classofrelations/deprecated.csv',header=None,names=['word'])

In [64]:
len(deprecated_relations)

209

In [65]:
deprecated_relations.head()

,word
0,% for
1,'s III be
2,)evaluate
3,#NAME?
4,#NAME?


In [66]:
deprecated_relations_dict={}
for i in deprecated_relations['word'].to_list():
    deprecated_relations_dict[i.lower()]=True

In [67]:
deprecated_relations_dict.get('have')

True

In [68]:
list(deprecated_relations_dict.items())[20]

('aforemention', True)

In [69]:
triplelistnewcopy[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [70]:
i

'promising'

In [71]:
bedict

{'am': 'be',
 'is': 'be',
 'are': 'be',
 'was': 'be',
 'were': 'be',
 'been': 'be',
 'have been': 'be',
 'has been': 'be'}

In [72]:
not_relations=pd.read_csv('../classofrelations/not_relations.csv',header=None,names=['word'])

In [73]:
len(not_relations)

61

In [74]:
not_relations.head()

,word
0,1p13 near
1,a of
2,abstractdiagnosing
3,acetylcholinesterase
4,apolipoprotein


In [75]:
not_relations_dict={}
for i in not_relations['word'].to_list():
    not_relations_dict[i.lower()]=True

In [76]:
list(not_relations_dict.items())[20]

('near 7q11 be', True)

In [77]:
triplelistnew=[]
for i in triplelistnewcopy: 
    if not deprecated_relations_dict.get(i.get('triple')[1].lower()) and not not_relations_dict.get(i.get('triple')[1].lower()):
        if i.get('triple')[1][0]=='\-':
            continue
        temp_triple=i.get('triple')[1].split(' ')
        for j,k in enumerate(temp_triple):
            if bedict.get(k.lower()):
                temp_triple[j]=bedict.get(k.lower())
        i['triple'][1]=' '.join(temp_triple)
        if i.get('triple')[1].lower()=='be' or bedict.get(i.get('triple')[1].lower()):
            continue
        if i.get('triple')[1].lower()=='means':
            i['triple'][1]='mean'
        if i.get('triple')[1].lower()=='meets':
            i['triple'][1]='meet'
        mid_judge=True
        for j in i.get('triple')[1].split(' '):
            if j.lower() not in i.get('triple')[0].lower().split(' ') and j.lower() not in i.get('triple')[2].lower().split(' '):
                pass
            else:
                mid_judge=False
        if mid_judge:
            triplelistnew.append(i)

In [78]:
len(triplelistnew)

145022

In [79]:
triplelistnew[10]

{'triple': ['Sarcopenia', 'associated', 'constipation'],
 'PMID': '34769606',
 'sentence': '(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.',
 'AB': '(1) Background: As the clinical relevance of constipation and sarcopenia is not well studied, we aimed to investigate the association between them in older adults. (2) Methods: A cross-sectional study was conducted on 1278 community-dwelling older adults in South Korea. The Rome IV criteria were used to identify patients with clinically defined constipation, while sarcopenia was defined by the Asian Working Group for Sarcopenia consensus. The cohort was classified into three groups: no constipation, self-reported constipation only, and clinically defined constipation. (3) Results: The presence of constipation was associated with sarcopenia and slow gait speed (p < 0.001). After adjustment for possible covariates, the association with sarcopenia attenuated, while that for slo

In [80]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [81]:
entity_set=set()
for i in triplelistnew:
    entity_set.add(i.get('entity_one_transformed'))
    entity_set.add(i.get('entity_two_transformed'))

In [82]:
len(entity_set)

6906

In [83]:
entitydict={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        if not entitydict.get(j.get('entity1')) and not entitydict.get(j.get('entity2')):
            entitydict[j.get('entity1')]=j.get('target1_type')
            entitydict[j.get('entity2')]=j.get('target2_type')
        else:
            if entitydict.get(j.get('entity1')):
                if re.search(j.get('target1_type'), entitydict.get(j.get('entity1'))) :
                    pass
                else:
                    entitydict[j.get('entity1')]+=";"+j.get('target1_type')
            else:
                entitydict[j.get('entity1')]=j.get('target1_type')
            if entitydict.get(j.get('entity2')):
                if re.search(j.get('target2_type'), entitydict.get(j.get('entity2'))) :
                    pass
                else:
                    entitydict[j.get('entity2')]+=";"+j.get('target2_type')
            else:
                entitydict[j.get('entity2')]=j.get('target2_type')

In [84]:
len(entitydict)

6906

In [85]:
entitydict_tolist={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        if not entitydict_tolist.get(j.get('entity1')) and not entitydict_tolist.get(j.get('entity2')):
            entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]
        else:
            if entitydict_tolist.get(j.get('entity1')):
                if j.get('target1_type') in entitydict_tolist.get(j.get('entity1')):
                    pass
                else:
                    entitydict_tolist[j.get('entity1')]+=[j.get('target1_type')]
            else:
                entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            if entitydict_tolist.get(j.get('entity2')):
                if j.get('target2_type') in  entitydict_tolist.get(j.get('entity2')) :
                    pass
                else:
                    entitydict_tolist[j.get('entity2')]+=[j.get('target2_type')]
            else:
                entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]

In [86]:
entitydict.get('Lipopolysaccharides')

'Carbohydrate;Lipid;Toxin'

In [87]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [88]:
len(entitydict)

6906

In [89]:
list(entitydict.items())[-30:-1]

[('HBP1', 'Gene'),
 ('TRPC3', 'Gene'),
 ('FBXL4', 'Gene'),
 ('CD248', 'Gene'),
 ('NME1', 'Gene'),
 ('NXPH2', 'Gene'),
 ('ITPA', 'Gene'),
 ('SLC12A2', 'Gene'),
 ('DOP1B', 'Gene'),
 ('DAPK2', 'Gene'),
 ('IFNL3', 'Gene'),
 ('F2R', 'Gene'),
 ('NUPR1', 'Gene'),
 ('COL7A1', 'Gene'),
 ('FCGR3B', 'Gene'),
 ('SIRPA', 'Gene'),
 ('CD48', 'Gene'),
 ('SLC6A5', 'Gene'),
 ('ERBB3', 'Gene'),
 ('COPS2', 'Gene'),
 ('LINC01194', 'RNA'),
 ('MT-TD', 'RNA'),
 ('rs1409112179', 'Mutation'),
 ('rs869312966', 'Mutation'),
 ('rs1390860312', 'Mutation'),
 ('rs132630304', 'Mutation'),
 ('Ape Diseases', 'Disease'),
 ('rs193922916', 'Mutation'),
 ('rs104893736', 'Mutation')]

In [90]:
for i,j in entitydict.items():
    if re.search(';',j):
        print({i:j})

{'Ceramides': 'Carbohydrate;Lipid'}
{'Ristocetin': 'Carbohydrate;Peptide'}
{'Lipopolysaccharides': 'Carbohydrate;Lipid;Toxin'}
{'Sphingomyelins': 'Carbohydrate;Lipid'}
{'Daptomycin': 'Lipid;Peptide'}
{'Vancomycin': 'Carbohydrate;Peptide'}
{'Bleomycin': 'Carbohydrate;Peptide'}
{'Glycosylphosphatidylinositols': 'Carbohydrate;Lipid'}
{'Abciximab': 'Peptide;Protein'}
{'Angiotensin II': 'Peptide;Protein'}
{'Angiotensins': 'Peptide;Protein'}
{'Psychosine': 'Carbohydrate;Lipid'}
{'Glycolipids': 'Carbohydrate;Lipid'}
{'Gangliosides': 'Carbohydrate;Lipid'}
{'G(M1) Ganglioside': 'Carbohydrate;Lipid'}
{'G(M3) Ganglioside': 'Carbohydrate;Lipid'}
{'Glycosphingolipids': 'Carbohydrate;Lipid'}
{'Phospholipid Ethers': 'Carbohydrate;Lipid'}
{'Glycopeptides': 'Carbohydrate;Peptide'}
{'Cilastatin, Imipenem Drug Combination': 'Lipid;Pharmaceutical Preparations'}
{'Phalloidine': 'Peptide;Toxin'}
{'Globosides': 'Carbohydrate;Lipid'}
{'Glucosylceramides': 'Carbohydrate;Lipid'}
{'Cerebrosides': 'Carbohydrate;L

In [91]:
entitynum={}
num=1
for i in entitydict.keys():
    entitynum[i]=num
    num+=1

In [92]:
entitynum.get('APOE')

243

In [93]:
for i in triplelistnew:
    i['verb']=i.get('triple')[1]

In [94]:
entity_countnum={}
relation_countnum={}
tripleWeight={}
tripleWeightunique=set()
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if (i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')) not in tripleWeightunique:
            tripleWeightunique.add((i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')))
            entity_countnum[i.get('entity_one_transformed')]=entity_countnum.get(i.get('entity_one_transformed'),0)+1
            entity_countnum[i.get('entity_two_transformed')]=entity_countnum.get(i.get('entity_two_transformed'),0)+1
            relation_countnum[i.get('triple')[1]]=relation_countnum.get(i.get('triple')[1],0)+1
            tripleWeight[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')),0)+1

In [95]:
len(entity_countnum)

6906

In [96]:
len(tripleWeight)

115514

In [97]:
len(triplelistnew)

145022

In [98]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [99]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
def cal_date(i):
    j=i
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    return date

In [100]:
for i in triplelistnew:
    if jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF']=jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF']=jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF5']=jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF5']=jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF5']=0.0

In [101]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [102]:
relation_interpolation_list=[]
relation_interpolation_dict={}
for i in triplelistnew:
    if (i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')) not in relation_interpolation_dict.keys():
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]={'from':entitynum.get(i.get('entity_one_transformed'))-1,
                                   'to':entitynum.get(i.get('entity_two_transformed'))-1,'relation':i.get('triple')[1],
                                   'PMID':[i.get('PMID')],'sentence':[i.get('sentence')],
                                   'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
                                   'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
                                    'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
                                    'IF':[i.get('IF')],   'IF5':[i.get('IF5')],   
                                    'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
                                    'first_entity_transformed':i.get('entity_one_transformed'),'second_entity_transformed':i.get('entity_two_transformed'),                                                     
                                    'first_entity':[i.get('entity_one')],'second_entity':[i.get('entity_two')],
                                    'first_type':entitydict_tolist.get(i.get('entity_one_transformed')),'second_type':entitydict_tolist.get(i.get('entity_two_transformed')),
                                     'method':[', '.join(i.get('method'))]
                                                                                                                           }
    else:
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['PMID']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('PMID')+[i.get('PMID')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['sentence']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('sentence')+[i.get('sentence')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['first_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('first_entity')+[i.get('entity_one')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['second_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('second_entity')+[i.get('entity_two')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['DP']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TI']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TA']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['date']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF')+[i.get('IF')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF5']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF5')+[i.get('IF5')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['method']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('method')+[', '.join(i.get('method'))]

In [103]:
list(relation_interpolation_dict.items())[10]

(('Sarcopenia', 'associated', 'Constipation'),
 {'from': 3,
  'to': 13,
  'relation': 'associated',
  'PMID': ['34769606'],
  'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
  'DP': ['2021 Oct 21'],
  'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
  'TA': ['Int J Environ Res Public Health'],
  'IF': [4.614],
  'IF5': [4.798],
  'date': [20211021],
  'first_entity_transformed': 'Sarcopenia',
  'second_entity_transformed': 'Constipation',
  'first_entity': ['Sarcopenia'],
  'second_entity': ['constipation'],
  'first_type': ['Disease'],
  'second_type': ['Disease'],
  'method': ['open domain']})

In [104]:
len(relation_interpolation_dict)

115522

In [105]:
relation_interpolation_list=list(relation_interpolation_dict.values())

In [106]:
len(relation_interpolation_list)

115522

In [107]:
entity2content={}
for i,j in enumerate(triplelistnew):
    if j.get('entity_one_transformed') in entity2content.keys():
        entity2content[j.get('entity_one_transformed')]['sentence']=entity2content.get(j.get('entity_one_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_one_transformed')]['PMID']=entity2content.get(j.get('entity_one_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_one_transformed')]['entity']=entity2content.get(j.get('entity_one_transformed')).get('entity')+[j.get('entity_one')]
        entity2content[j.get('entity_one_transformed')]['DP']=entity2content.get(j.get('entity_one_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_one_transformed')]['TI']=entity2content.get(j.get('entity_one_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_one_transformed')]['TA']=entity2content.get(j.get('entity_one_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_one_transformed')]['date']=entity2content.get(j.get('entity_one_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_one_transformed')]['IF']=entity2content.get(j.get('entity_one_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_one_transformed')]['IF5']=entity2content.get(j.get('entity_one_transformed')).get('IF5')+[j.get('IF5')]
        entity2content[j.get('entity_one_transformed')]['method']=entity2content.get(j.get('entity_one_transformed')).get('method')+[', '.join(j.get('method'))]
    else:
        entity2content[j.get('entity_one_transformed')]={'entity_transformed':j.get('entity_one_transformed'),'entity':[j.get('entity_one')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')],'method':[', '.join(j.get('method'))]}
    if j.get('entity_two_transformed') in entity2content.keys():
        entity2content[j.get('entity_two_transformed')]['sentence']=entity2content.get(j.get('entity_two_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_two_transformed')]['PMID']=entity2content.get(j.get('entity_two_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_two_transformed')]['entity']=entity2content.get(j.get('entity_two_transformed')).get('entity')+[j.get('entity_two')]
        entity2content[j.get('entity_two_transformed')]['DP']=entity2content.get(j.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_two_transformed')]['TI']=entity2content.get(j.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_two_transformed')]['TA']=entity2content.get(j.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_two_transformed')]['date']=entity2content.get(j.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_two_transformed')]['IF']=entity2content.get(j.get('entity_two_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_two_transformed')]['IF5']=entity2content.get(j.get('entity_two_transformed')).get('IF5')+[j.get('IF5')]
        entity2content[j.get('entity_two_transformed')]['method']=entity2content.get(j.get('entity_two_transformed')).get('IF5')+[', '.join(j.get('method'))]
    else:
        entity2content[j.get('entity_two_transformed')]={'entity_transformed':j.get('entity_two_transformed'),'entity':[j.get('entity_two')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')],'method':[', '.join(j.get('method'))]}

In [108]:
len(entity2content)

6906

In [109]:
entity2list=list(entity2content.values())

In [110]:
len(entity2list)

6906

In [111]:
for i in entity2list:
    i['_id']=i.get('entity_transformed')

In [112]:
entitydict.get(entity2list[0].get('entity_transformed'))

'Disease'

In [113]:
for i in entity2list:
    i['type']=entitydict.get(i.get('entity_transformed')).split(';')

In [114]:
len(entity2list)

6906

In [346]:
json.dump(entity2list,open('results/step4/entityinfo.json','w'))

In [115]:
for i in entity2list:
    if i.get('entity_transformed')=='HMGB1':
        print(i)

{'entity_transformed': 'HMGB1', 'entity': ['HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1'], 'sentence': ['In an in vitro study we used the 18-beta-stereoisomer of GTA to enhance Sirt6 expression levels, inhibiting through this mechanism the translocation of HMGB1 protein from nucleus and reversing its extracellular accumulation stimulated by lipopolysaccharides.', 'In an in vitro study we used the 18-beta-stereoisomer of GTA to enhance Sirt6 expression levels, inhibiting through this mechanism the translocation of HMGB1 protein from nucleus and reversing its extracellular accumulation stimulated by lipopolysaccharides.', 'In an in vitro study we used the 18-beta-stereoisomer of GTA to enhance Sirt6 expression levels, inhibiting through this mechanism the translocation of HMGB1 protein from nucleus and r

In [116]:
entity2list[-1]

{'entity_transformed': 'rs120074195',
 'entity': ['V307L', 'V307L', 'V307L', 'V307L'],
 'sentence': ["This simulation study investigated arrhythmia dynamics in multi-scale human ventricle models associated with the SQT2-related V307L KCNQ1 'gain-of-function' mutation, which increases slow-delayed rectifier potassium current (IKs).",
  "This simulation study investigated arrhythmia dynamics in multi-scale human ventricle models associated with the SQT2-related V307L KCNQ1 'gain-of-function' mutation, which increases slow-delayed rectifier potassium current (IKs).",
  "This simulation study investigated arrhythmia dynamics in multi-scale human ventricle models associated with the SQT2-related V307L KCNQ1 'gain-of-function' mutation, which increases slow-delayed rectifier potassium current (IKs).",
  'A Markov chain (MC) model recapitulating wild type (WT) and V307L mutant IKs kinetics was incorporated into a model of the human ventricular action potential (AP) for investigation of QT int

In [117]:
for i in entity2list:
    if i.get('entity_transformed')=='Lipopolysaccharides':
        print(i.get('type'))

['Carbohydrate', 'Lipid', 'Toxin']


In [118]:
list(relation_interpolation_dict.items())[:3]

[(('Pulmonary Disease, Chronic Obstructive', 'defined', 'Inflammation'),
  {'from': 0,
   'to': 1,
   'relation': 'defined',
   'PMID': ['30781849'],
   'sentence': ['(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.'],
   'DP': ['2019 Feb 13'],
   'TI': ['Chronic Obstructive Pulmonary Disease as a Main Factor of Premature Aging.'],
   'TA': ['Int J Environ Res Public Health'],
   'IF': [4.614],
   'IF5': [4.798],
   'date': [20190213],
   'first_entity_transformed': 'Pulmonary Disease, Chronic Obstructive',
   'second_entity_transformed': 'Inflammation',
   'first_entity': ['COPD'],
   'second_entity': ['inflammatory disorder'],
   'first_type': ['Disease'],
   'second_type': ['Disease'],
   'method': ['deep learning, shortest path']}),
 (('Anorexia', 'associate', 'Sarcopenia'),
  {'from': 2,
   'to': 3,
   'relation': 'associate',
   'PMID': ['30641897'],
   'sentence': ["(1) Bac

In [119]:
relation_interpolation_list[10]

{'from': 3,
 'to': 13,
 'relation': 'associated',
 'PMID': ['34769606'],
 'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
 'DP': ['2021 Oct 21'],
 'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
 'TA': ['Int J Environ Res Public Health'],
 'IF': [4.614],
 'IF5': [4.798],
 'date': [20211021],
 'first_entity_transformed': 'Sarcopenia',
 'second_entity_transformed': 'Constipation',
 'first_entity': ['Sarcopenia'],
 'second_entity': ['constipation'],
 'first_type': ['Disease'],
 'second_type': ['Disease'],
 'method': ['open domain']}

In [120]:
len(relation_interpolation_list)

115522

In [337]:
json.dump(relation_interpolation_list,open('results/step4/relationinfo.json','w'))

In [121]:
relation_interpolation_list[100]

{'from': 5,
 'to': 96,
 'relation': 'associated',
 'PMID': ['34772307'],
 'sentence': ['Drowsiness/lethargy (OR = 1.48, 95% CI: 1.22-1.82), agitation (OR = 1.66, 95% CI: 1.11-2.50), coma (OR = 23.95, 95% CI: 17.05-33.64), bradycardia (OR = 2.29, 95% CI: 1.22-4.32), rhabdomyolysis (OR = 8.84, 95% CI: 3.71-21.03), hypothermia (OR = 4.1, 95% CI: 1.77-9.51), and hyperthermia 2.10 (OR = 2.10, 95% CI: 1.04-4.22) were likely associated with major outcomes or death.'],
 'DP': ['2021 Dec'],
 'TI': ['Prognostic factors of acetaminophen exposure in the United States: An analysis of 39,000 patients.'],
 'TA': ['Hum Exp Toxicol'],
 'IF': [3.247],
 'IF5': [3.045],
 'date': [20211201],
 'first_entity_transformed': 'Death',
 'second_entity_transformed': 'Rhabdomyolysis',
 'first_entity': ['death'],
 'second_entity': ['rhabdomyolysis'],
 'first_type': ['Disease'],
 'second_type': ['Disease'],
 'method': ['shortest path']}

In [122]:
len(relation_interpolation_list)

115522

In [123]:
len(triplelistnew)

145022

In [104]:
csvf_entity = open("results/step4/entity.csv", "w", newline='', encoding='utf-8')
w_entity = csv.writer(csvf_entity)

In [105]:
w_entity.writerow(("entity:ID", "name","type","frequency",":LABEL"))

38

In [106]:
for i,j in entitydict.items():
    w_entity.writerow((entitynum.get(i), i,j,int(entity_countnum.get(i)),j))

In [107]:
csvf_entity.close()

In [124]:
entitycsv=pd.read_csv('results/step4/entity.csv')

In [125]:
entitycsv[:3]

,entity:ID,name,type,frequency,:LABEL
0,1,"Pulmonary Disease, Chronic Obstructive",Disease,1004,Disease
1,2,Inflammation,Disease,4106,Disease
2,3,Anorexia,Disease,140,Disease


In [126]:
len(entitycsv)

6906

In [111]:
csvf_entity = open("results/step4/roles.csv", "w", newline='', encoding='utf-8')
w_entity = csv.writer(csvf_entity)

In [112]:
w_entity.writerow((':START_ID', ':END_ID', "relation", "weight", "method", ":TYPE" ))

48

In [127]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [ ]:
entitynotsame={}
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if not entitynotsame.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))) :
            w_entity.writerow((  entitynum.get(i.get('entity_one_transformed')),entitynum.get(i.get('entity_two_transformed')), i.get('triple')[1].lower(),int(tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')))),'; '.join(i.get('method')), i.get('triple')[1]   ))
        else:
            continue
        entitynotsame[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=True

In [129]:
entitynum.get('APOE')

243

In [116]:
csvf_entity.close()

In [117]:
for i in triplelistnew:
    if i.get('sentence')=='Incidence of epilepsy was not statistically elevated among males, those with dementia, or individuals reporting a history of head injury or treatment for depression.':
        print(i)

{'triple': ['dementia', 'report', 'head injury'], 'PMID': '16870396', 'sentence': 'Incidence of epilepsy was not statistically elevated among males, those with dementia, or individuals reporting a history of head injury or treatment for depression.', 'AB': 'OBJECTIVE: To determine age-specific incidence and cumulative incidence of epilepsy in a well-defined cohort of elderly people, and to examine how rates of epilepsy are modified by sex, race, stroke, dementia, head injury, and depression. METHODS: The authors examined data from a reconstructed cohort based on 1919 community-dwelling volunteers, followed as part of a large ongoing prospective aging study. RESULTS: Age-specific incidence was 10.6 (per 100,000 person-years) between ages 45 and 59, 25.8 between ages 60 and 74, and 101.1 between ages 75 and 89. Cumulative incidence was 0.15% from age 45 to age 60, 0.38% to age 70, 1.01% to age 80, and 1.47% to age 90. In addition, the difference in cumulative incidence among African-Amer

In [130]:
roles=pd.read_csv("results/step4/roles.csv")

In [131]:
len(roles)

115514

In [132]:
roles.head()

,:START_ID,:END_ID,relation,weight,method,:TYPE
0,1,2,defined,1,deep learning; shortest path,defined
1,3,4,associate,1,deep learning,associate
2,4,3,associate,1,deep learning,associate
3,5,6,recognized,1,deep learning,recognized
4,5,6,increase,1,shortest path,increase


In [133]:
relation_list=roles.relation.to_list()

In [134]:
type(relation_list)

list

In [135]:
relation_list[:3]

['defined', 'associate', 'associate']

In [136]:
len(relation_list)

115514

In [137]:
len(set(relation_list))

3013

In [138]:
relation_list_set=pd.read_csv('../classofrelations/relation_list_set.csv',header=None,names=['word'])

In [139]:
relation_list_set_dict={}
for i in relation_list_set['word'].to_list():
    relation_list_set_dict[i.lower()]=True

In [140]:
positive_relations=pd.read_csv('../classofrelations/positive-relations.csv',header=None,names=['word'])

In [141]:
positive_relations_dict={}
for i in positive_relations['word'].to_list():
    positive_relations_dict[i.lower()]=True

In [142]:
negative_relations=pd.read_csv('../classofrelations/negative-relations.csv',header=None,names=['word'])

In [143]:
negative_relations_dict={}
for i in negative_relations['word'].to_list():
    negative_relations_dict[i.lower()]=True

In [144]:
negative_relations['word'].to_list()[-5:-1]

['rescued', 'restrained', 'reduce complications in', 'reduce relative in']

In [145]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [ ]:
dropout=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='positive'
                elif negative_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='negative'
                else:
                    i['partofspeech']='none'
                    dropout.append(i)

In [ ]:
len(dropout)

9748

In [148]:
relation_list=[]
disease_list_raw=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                relation_list.append(i.get('triple')[1].lower())
                disease_list_raw.append(i)

In [149]:
len(relation_list)

19540

In [150]:
len(disease_list_raw)

19540

In [151]:
new_relation=set()
for i in disease_list_raw:
    if not relation_list_set_dict.get(i.get('triple')[1].lower()):
        new_relation.add(i.get('triple')[1].lower())

In [152]:
len(new_relation)

0

In [154]:
for i in disease_list_raw:
    if i.get('triple')[1].lower()=='with those be':
        print(i.get('triple'))
        print(i.get('sentence'))
        # break

['CD4', 'with those be', 'neuroretinal disease']
RESULTS: Ocular manifestations from any cause develop from 35 years of age in patients with HIV/AIDS, with the highest risk for age-related macular degeneration over the fourth decade of life and for the development of neuroretinal disorder on the fifth decade of life; some studies report a slight tendency to diagnose macular degeneration in women and those who acquired AIDS through sexual contact; data contrasted with increased risk for diagnosing neuroretinal disorder in homosexual men who also use intravenous drugs, possibly due to oversampling in studies; non-Hispanic whites and African Americans were the races most commonly affected by neuroretinal disease; the means between the 11.3-14.5 years elapsed since the HIV diagnosis were more frequently associated with cognitive impairment and both in those with high or low CD4 counts, and in patients with high or low viral loads, neuroretinal disease without Statistically significant diff

In [155]:
new_relation

set()

In [156]:
len(relation_list_set['word'].to_list())

1721

In [142]:
relation_list_set_new=sorted(relation_list_set['word'].to_list())
for i in new_relation:
    relation_list_set_new.append(i)
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('../classofrelations/relation_list_set.csv')

In [143]:
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('results/step4/relation_list_set.csv')

# DEAL WITH new_relation!!!!!!!!!!!!
## Classify the new_relation list into posit, negat, and middle lists.

In [214]:
posit=['characterise','culminate','forecast','imply','intensify', 'intercorrelated','misregulated',\
      'overproduced','portrayed',]
negat=['advise','antagonize','avert','be actionable target in', 'be important genetic marker for',\
       'be important promoter in', 'be significant prognostic factors for','cease','chemosensitize',\
      'constrained', 'decelerate', 'deferred','eradicate','halted','hindered','improve quality in',\
      'reactivate','rescued','restrained','reduce complications in','reduce relative in',\
      'show activity without',]
# middle=['blocked','drifting','engage','expanded','hyperactivated','incorporated','mistaken','processed',
#        'reformed','relapsed','span','supplemented']

In [215]:
for i in disease_list_raw:
    if i.get('triple')[1]=='supplemented':
        print(i)
        break

{'triple': ['PUFA', 'supplemented', 'osteoporosis'], 'PMID': '12020975', 'sentence': 'OBJECTIVE: To review the effect of a diet supplemented with polyunsaturated fatty acids (PUFA) on prevention or treatment of osteoporosis.', 'AB': 'OBJECTIVE: To review the effect of a diet supplemented with polyunsaturated fatty acids (PUFA) on prevention or treatment of osteoporosis. METHODS: MEDLINE (1966-April 2001), Allied Complementary Medicine (1985-2001), Cochrane Library and Database of Systematic Reviews (1st Quarter 2001) was searched. Five reviews and no systematic reviews were found on this topic in the Cochrane Library. Eleven relevant in-vivo studies were identified on the effect of these compounds on bone. Eight were animal studies and three were randomised control trials (RCT) in human. RESULTS: There are two classes of PUFA designated as n-3 and n-6 with alpha-linolenic acid (ALA). These two different types of PUFA differently influence prostaglandin formation and hence modulate bone

In [158]:
positive_relations_new=positive_relations['word'].to_list()
negative_relations_new=negative_relations['word'].to_list()

In [ ]:
positive_relations_new=positive_relations['word'].to_list()
for i in posit:
    positive_relations_new.append(i)
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('../classofrelations/positive-relations.csv')

In [217]:
negative_relations_new=negative_relations['word'].to_list()
for i in negat:
    negative_relations_new.append(i)
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('../classofrelations/negative-relations.csv')

In [218]:
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('results/step4/positive-relations.csv')
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('results/step4/negative-relations.csv')

In [159]:
print(len(positive_relations_new))
print(len(negative_relations_new))

328
271


In [160]:
negative_relations['word'].to_list()[-5:-1]

['rescued', 'restrained', 'reduce complications in', 'reduce relative in']

In [161]:
positive_relations_new_dict={}
for i in positive_relations_new:
    positive_relations_new_dict[i.lower()]=True

In [162]:
negative_relations_new_dict={}
for i in negative_relations_new:
    negative_relations_new_dict[i.lower()]=True

In [163]:
dropout=[]
disease_list=[]
for i in disease_list_raw:
    if positive_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='positive'
        disease_list.append(i)
    elif negative_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='negative'
        disease_list.append(i)
    else:
        i['partofspeech']='none'
        dropout.append(i)

In [164]:
len(disease_list)

9792

In [165]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in disease_list:
    j=pubtator_info_dict.get(i.get('PMID')).get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['TA']=pubtator_info_dict.get(i.get('PMID')).get('TA')
    i['JT']=pubtator_info_dict.get(i.get('PMID')).get('JT')
    i['year']=int(str(date)[:4])

In [166]:
disease_list[-1]

{'triple': ['G510S', 'associate', 'systemic lupus erythematosus'],
 'PMID': '36961448',
 'sentence': 'We introduced the Prkcd G510S mutation that we previously associated to a Mendelian cause of systemic lupus erythematosus in the mouse genome, using CRISPR-Cas9 gene editing.',
 'AB': 'Autosomal recessive PRKCD deficiency has previously been associated with the development of systemic lupus erythematosus in human patients, but the mechanisms underlying autoimmunity remain poorly understood. We introduced the Prkcd G510S mutation that we previously associated to a Mendelian cause of systemic lupus erythematosus in the mouse genome, using CRISPR-Cas9 gene editing. PrkcdG510S/G510S mice recapitulated the human phenotype and had reduced lifespan. We demonstrate that this phenotype is linked to a B cell-autonomous role of Prkcd. A detailed analysis of B cell activation in PrkcdG510S/G510S mice shows an upregulation of the PI3K/mTOR pathway after the engagement of the BCR in these cells, lea

In [167]:
for i in disease_list:
    if i.get('triple')[1]=='impede':
        print(i)
        break

{'triple': ['NOTCH1', 'impede', 'carcinogenesis'], 'PMID': '36658434', 'sentence': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis.', 'AB': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis. Here we test this hypothesis. Sequencing NOTCH1 mutant clones in aging human esophagus reveals frequent biallelic mutations that block NOTCH1 signaling. In mouse esophagus, heterozygous Notch1 mutation confers a competitive advantage over wild-type cells, an effect enhanced by loss of the second allele. Widespread Notch1 loss alters transcription but has minimal effects on the epithelial structure and cell dynamics. In a carcinogenesis model, Notch1 mutations were less prevalent in tu

In [168]:
len(disease_list)

9792

In [169]:
entity_pairs=['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [170]:
names=locals()
for i in entity_pairs:
    names[i+'_typecount']=set()
for j in disease_list:
    for i in entitydict_tolist.get(j.get('entity_one_transformed')):
        i=i.replace(' ','_')
        names[i+'_typecount'].add(j.get('entity_one_transformed'))

In [171]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [172]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_typecount']))))

Gene 1473
RNA 43
Protein 17
Carbohydrate 81
Lipid 84
Peptide 26
Pharmaceutical_Preparations 3
Toxin 3
Mutation 293
Disease 0


In [173]:
disease_list_copy=copy.deepcopy(disease_list)

In [174]:
disease_list_copy[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [175]:
for i in disease_list_copy:
    if len(entitydict_tolist.get(i.get('entity_one_transformed')))>1:
        i['entity_one_type']='; '.join(entitydict_tolist.get(i.get('entity_one_transformed')))

In [176]:
list(entitydict_tolist.items())[0]

('Pulmonary Disease, Chronic Obstructive', ['Disease'])

In [177]:
disease_list_copy[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [178]:
len(disease_list_copy)

9792

In [179]:
positive_relations=[]
negative_relations=[]
for i in disease_list_copy:
    if i.get('partofspeech')=='positive':
        positive_relations.append(i)
    else:
        negative_relations.append(i)

In [180]:
len(positive_relations)

8330

In [241]:
json.dump(positive_relations,open('results/step4/aging_relations.json','w'))
json.dump(negative_relations,open('results/step4/longevity_relations.json','w'))

In [242]:
json.dump(disease_list_copy,open('results/step4/disease_relation.json','w'))

In [3]:
positive_relations=json.load(open('results/step4/aging_relations.json','r'))
negative_relations=json.load(open('results/step4/longevity_relations.json','r'))
disease_list_copy=json.load(open('results/step4/disease_relation.json','r'))

# Statistics

In [181]:
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_aging']=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_aging'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_aging'])))

Gene	1353
RNA	39
Protein	17
Carbohydrate	70
Lipid	75
Peptide	22
Pharmaceutical Preparations	2
Toxin	3
Mutation	289
Disease	0


In [182]:
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(str(len(names[j+'_aging']))+',', end='')

1353,39,17,70,75,22,2,3,289,0,

In [183]:
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_longevity']=set()
for i in disease_list:
    if i.get('partofspeech')=='negative':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_longevity'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_longevity'])))

Gene	396
RNA	12
Protein	8
Carbohydrate	52
Lipid	40
Peptide	12
Pharmaceutical Preparations	3
Toxin	1
Mutation	8
Disease	0


In [184]:
Genedict=json.load(open('../dict/Genedict.json'))

In [185]:
list(Genedict.items())[:10]

[('A1BG', ['A1BG']),
 ('A1CF', ['A1CF']),
 ('A2M', ['A2M']),
 ('A2ML1', ['A2ML1']),
 ('A3GALT2', ['A3GALT2']),
 ('A4GALT', ['A4GALT']),
 ('A4GNT', ['A4GNT']),
 ('AAAS', ['AAAS']),
 ('AACS', ['AACS']),
 ('AADAC', ['AADAC'])]

In [186]:
Genedict_upper={}
for i,j in Genedict.items():
    Genedict_upper[i.upper()]=j

In [187]:
num_of_aging=set()
num_of_longevity=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        num_of_aging.add(i.get('entity_one_transformed'))
    elif i.get('partofspeech')=='negative':
        num_of_longevity.add(i.get('entity_one_transformed'))
print(len(num_of_aging))
print(len(num_of_longevity))

1855
525


In [188]:
disease_entity_list=[]
disease_entity_triple=set()
disease_entity_dict={}
for i in disease_list:
    if i.get('partofspeech')=='positive':
        if i.get('entity_one_type')=='Gene':
            disease_entity_triple.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list.append(i)
            disease_entity_dict[i.get('entity_one_transformed')]=disease_entity_dict.get(i.get('entity_one_transformed'),0)+1

In [189]:
len(disease_entity_dict)

1353

In [190]:
disease_entity_dict_sorted = sorted(disease_entity_dict.items(), key=lambda x: x[1], reverse=True)

In [191]:
len(disease_entity_dict_sorted)

1353

In [192]:
for i in disease_entity_dict_sorted[:20]:
    print(i)

('INS', 474)
('CRP', 239)
('APOE', 180)
('KL', 175)
('MAPT', 163)
('SIRT1', 103)
('IL6', 88)
('ADIPOQ', 80)
('CD4', 74)
('LEP', 70)
('LMNA', 65)
('ALB', 64)
('WRN', 62)
('TP53', 56)
('SIRT6', 46)
('ACE', 43)
('MTOR', 42)
('CST3', 41)
('SHBG', 39)
('BDNF', 36)


In [193]:
relation_interpolation_list[0]

{'from': 0,
 'to': 1,
 'relation': 'defined',
 'PMID': ['30781849'],
 'sentence': ['(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.'],
 'DP': ['2019 Feb 13'],
 'TI': ['Chronic Obstructive Pulmonary Disease as a Main Factor of Premature Aging.'],
 'TA': ['Int J Environ Res Public Health'],
 'IF': [4.614],
 'IF5': [4.798],
 'date': [20190213],
 'first_entity_transformed': 'Pulmonary Disease, Chronic Obstructive',
 'second_entity_transformed': 'Inflammation',
 'first_entity': ['COPD'],
 'second_entity': ['inflammatory disorder'],
 'first_type': ['Disease'],
 'second_type': ['Disease'],
 'method': ['deep learning, shortest path']}

In [194]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [195]:
direction=set()
direction_list=[]
direction_type=[]
for i in triplelistnew:
    for j in entitydict_tolist.get(i.get('entity_one_transformed')):
        for k in entitydict_tolist.get(i.get('entity_two_transformed')):
            if (i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k) not in direction:
                direction.add((i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k))
                direction_list.append((i.get('entity_one_transformed'),(i.get('entity_two_transformed'))))
                direction_type.append((j.replace(' ','_'),k.replace(' ','_')))

In [196]:
len(direction_list)

59822

In [197]:
type_dict={}
type_dict_ensure={}
for i in range(len(direction_list)):
    if not type_dict_ensure.get((direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])):
        type_dict[(direction_list[i][0],direction_list[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))
        type_dict_ensure[(direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))

In [198]:
len(type_dict)

59313

In [199]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [200]:
entity_pairs=['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [201]:
entity_pairs_dict={}
entity_pairs_dict_num={}
for i in entity_pairs:
    temp={}
    temp2={}
    for j in entity_pairs:
        temp.update({j:0})
        temp2.update({j:set()})
    entity_pairs_dict_num[i]=temp
    entity_pairs_dict[i]=temp2

In [202]:
entity_pairs_dict

{'Gene': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'RNA': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Protein': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Carbohydrate': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Lipid': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipi

In [203]:
for i in range(len(direction_list)):
        if direction_list[i][0] not in entity_pairs_dict.get(direction_type[i][1]).get(direction_type[i][0]):
            entity_pairs_dict[direction_type[i][1]][direction_type[i][0]].add(direction_list[i][0])
            entity_pairs_dict_num[direction_type[i][1]][direction_type[i][0]]+=1

In [204]:
entity_pairs_dict_num

{'Gene': {'Gene': 1240,
  'RNA': 39,
  'Protein': 10,
  'Carbohydrate': 56,
  'Lipid': 47,
  'Peptide': 24,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 2,
  'Mutation': 233,
  'Disease': 801},
 'RNA': {'Gene': 41,
  'RNA': 6,
  'Protein': 0,
  'Carbohydrate': 1,
  'Lipid': 0,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 1,
  'Disease': 39},
 'Protein': {'Gene': 37,
  'RNA': 0,
  'Protein': 4,
  'Carbohydrate': 5,
  'Lipid': 4,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 0,
  'Disease': 82},
 'Carbohydrate': {'Gene': 167,
  'RNA': 3,
  'Protein': 3,
  'Carbohydrate': 69,
  'Lipid': 38,
  'Peptide': 10,
  'Pharmaceutical_Preparations': 2,
  'Toxin': 1,
  'Mutation': 7,
  'Disease': 325},
 'Lipid': {'Gene': 186,
  'RNA': 5,
  'Protein': 2,
  'Carbohydrate': 28,
  'Lipid': 69,
  'Peptide': 6,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 1,
  'Mutation': 16,
  'Disease': 335},
 'Peptide': {'Gene': 53,
  'RNA': 0,
  

In [205]:
countnum=0
for i in entity_pairs_dict_num.values():
    for j in i.values():
        countnum+=j
print(countnum)

9757


In [206]:
relation_display=[]
for i,j in entity_pairs_dict_num.items():
    for k,l in j.items():
        relation_display.append([k,i+' ',l])

In [271]:
j

{'Gene': 2046,
 'RNA': 65,
 'Protein': 23,
 'Carbohydrate': 109,
 'Lipid': 112,
 'Peptide': 38,
 'Pharmaceutical_Preparations': 8,
 'Toxin': 5,
 'Mutation': 398,
 'Disease': 2329}

In [272]:
relation_display

[['Gene', 'Gene ', 1240],
 ['RNA', 'Gene ', 39],
 ['Protein', 'Gene ', 10],
 ['Carbohydrate', 'Gene ', 56],
 ['Lipid', 'Gene ', 47],
 ['Peptide', 'Gene ', 24],
 ['Pharmaceutical_Preparations', 'Gene ', 0],
 ['Toxin', 'Gene ', 2],
 ['Mutation', 'Gene ', 233],
 ['Disease', 'Gene ', 801],
 ['Gene', 'RNA ', 41],
 ['RNA', 'RNA ', 6],
 ['Protein', 'RNA ', 0],
 ['Carbohydrate', 'RNA ', 1],
 ['Lipid', 'RNA ', 0],
 ['Peptide', 'RNA ', 0],
 ['Pharmaceutical_Preparations', 'RNA ', 0],
 ['Toxin', 'RNA ', 0],
 ['Mutation', 'RNA ', 1],
 ['Disease', 'RNA ', 39],
 ['Gene', 'Protein ', 37],
 ['RNA', 'Protein ', 0],
 ['Protein', 'Protein ', 4],
 ['Carbohydrate', 'Protein ', 5],
 ['Lipid', 'Protein ', 4],
 ['Peptide', 'Protein ', 0],
 ['Pharmaceutical_Preparations', 'Protein ', 0],
 ['Toxin', 'Protein ', 0],
 ['Mutation', 'Protein ', 0],
 ['Disease', 'Protein ', 82],
 ['Gene', 'Carbohydrate ', 167],
 ['RNA', 'Carbohydrate ', 3],
 ['Protein', 'Carbohydrate ', 3],
 ['Carbohydrate', 'Carbohydrate ', 69],
 [

In [273]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [274]:
for i in entity_pairs:
    xs=[]
    for j in relation_display:
        if j[0]==i.replace(' ','_'):
            xs.append(str(j[-1]))
    print('\t'.join(xs))

1240	41	37	167	186	53	1	24	142	2046
39	6	0	3	5	0	0	2	1	65
10	0	4	3	2	1	0	0	0	23
56	1	5	69	28	5	0	0	2	109
47	0	4	38	69	12	0	2	8	112
24	0	0	10	6	9	0	1	1	38
0	0	0	2	0	1	0	0	0	8
2	0	0	1	1	1	0	2	0	5
233	1	0	7	16	1	0	0	94	398
801	39	82	325	335	110	14	31	161	2329


In [207]:
all_es=json.load(open('results/step2/all_es_lack.json'))

In [208]:
entity2aging={}
entity2longevity={}
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_new_dict.get(i.get('triple')[1].lower()):
                    entity2aging[i.get('entity_one_transformed')]=True
                elif negative_relations_new_dict.get(i.get('triple')[1].lower()):
                    entity2longevity[i.get('entity_one_transformed')]=True

In [209]:
len(entity2aging)

1855

In [210]:
len(entity2longevity)

525

In [211]:
aaaa=0
for i in entity2aging.keys():
    if entity2longevity.get(i):
        aaaa+=1
print(aaaa)

373


In [212]:
for i in all_es:
    if entitydict.get(i.get('entity')):
        i['network']=True
    else:
        i['network']=False
    if entity2aging.get(i.get('entity')):
        i['aging']=True
    else:
        i['aging']=False
    if entity2longevity.get(i.get('entity')):
        i['longevity']=True
    else:
        i['longevity']=False

In [213]:
all_es[-1]

{'entity': 'Shock, Traumatic',
 'PMID': ['15995458'],
 'literature': 1,
 'allyears': [2005],
 'type': 'Disease',
 'MeshID': 'D012774',
 'url': 'https://www.ncbi.nlm.nih.gov/mesh/68012774',
 'Summary': 'Shock produced as a result of trauma.',
 '_id': 'Disease_Shock, Traumatic',
 'datest': 20050601,
 'allwords': ['TRAUMATIC SHOCK'],
 'network': True,
 'aging': False,
 'longevity': False}

In [214]:
json.dump(all_es,open('results/step4/all_es.json','w'))

In [215]:
len(all_es)

12283

In [567]:
entity_es_set=set()
for i in all_es:
    entity_es_set.add(i.get('entity'))

In [568]:
len(list(entity_es_set))

12227

In [569]:
names=locals()
for i in entity_pairs:
    names[i+'_all']=[]
    for j in all_es:
        if j.get('type')==i.replace('_',' '):
            names[i+'_all'].append(j)

In [570]:
len(Gene_all)

5618

In [287]:
len(Disease_all)

3498

In [288]:
Gene_all_sorted=sorted(Gene_all,key=lambda x:x['literature'],reverse=True)

In [289]:
for i in entity_pairs:
    print('\n')
    print(i)
    names[i+'_all_sorted']=sorted(names[i+'_all'],key=lambda x:x['literature'],reverse=True)
    names[i+'_all_sorted_num']=0
    for j in names[i+'_all_sorted'][:10]:
        print("'"+j.get('entity')+"',")
for i in entity_pairs:
    print('\n')
    print(i)
    for j in names[i+'_all_sorted'][:10]:
        print(str(j.get('literature'))+",")
        names[i+'_all_sorted_num']+=j.get('literature')
for i in entity_pairs:
    print('\n')
    print(i)
    cxv=0
    for j in names[i+'_all_sorted'][:10]:
        z=j.get('literature')*100/names[i+'_all_sorted_num']
        print(str(round(z,2))+",")



Gene
'INS',
'CRP',
'IL6',
'APOE',
'CD4',
'TNF',
'MAPT',
'ALB',
'SIRT1',
'CD8A',


RNA
'H19',
'MIR34A',
'TERC',
'MIR21',
'MIR146A',
'MIR155',
'MIR132',
'MIR221',
'HCCAT5',
'MIR31',


Protein
'Infliximab',
'Rituximab',
'Bevacizumab',
'Denosumab',
'Trastuzumab',
'Nivolumab',
'Cetuximab',
'Palivizumab',
'Ipilimumab',
'Ranibizumab',


Carbohydrate
'Glucose',
'Ascorbic Acid',
'Hyaluronic Acid',
'Heparin',
'Doxorubicin',
'Sugars',
'Polysaccharides',
'Lipopolysaccharides',
'Digoxin',
'Fluorodeoxyglucose F18',


Lipid
'Cholesterol',
'Triglycerides',
'Fatty Acids',
'Fatty Acids, Omega-3',
'Fatty Acids, Unsaturated',
'Lipofuscin',
'Lipopolysaccharides',
'Phospholipids',
'Cholecalciferol',
'Docosahexaenoic Acids',


Peptide
'Glutathione',
'Cyclosporine',
'Vancomycin',
'Thyrotropin',
'Bleomycin',
'Ghrelin',
'Enalapril',
'Teriparatide',
'Glutathione Disulfide',
'Insulin Glargine',


Pharmaceutical_Preparations
'Trimethoprim, Sulfamethoxazole Drug Combination',
'Amoxicillin-Potassium Clavulanate Co

In [290]:
for i in entity_pairs:
    print('\n')
    print(i)
    print(names[i+'_all_sorted_num'])



Gene
12553


RNA
323


Protein
881


Carbohydrate
6777


Lipid
7561


Peptide
1489


Pharmaceutical_Preparations
186


Toxin
315


Mutation
259


Disease
145764


In [291]:
for i in entity_pairs:
    print(i)
    print(len(names[i+'_all']))


Gene
5618
RNA
385
Protein
43
Carbohydrate
211
Lipid
199
Peptide
89
Pharmaceutical_Preparations
15
Toxin
21
Mutation
2204
Disease
3498


In [292]:
for i in entity_pairs:
    print(i)
    axc=set()
    for j in names[i+'_all']:
        for k in j.get('PMID'):
            axc.add(k)
    print(len(axc))


Gene
31519
RNA
761
Protein
932
Carbohydrate
9348
Lipid
8430
Peptide
1894
Pharmaceutical_Preparations
171
Toxin
324
Mutation
1794
Disease
174576


In [6]:
all_es_result=json.load(open('results/step2/all_es_result.json'))

In [294]:
for i in entity_pairs:
    names[i.replace(' ','_')+'_sentences']=set()
for i in all_es_result:
    for j in i.get('sentence'):
        for k in j:
            if re.search('RNA',i.get('type')):
                names['RNA'.replace(' ','_')+'_sentences'].add(k)
            else:
                names[i.get('type').replace(' ','_')+'_sentences'].add(k)

In [295]:
for i in entity_pairs:
    print(i)
    print(len(names[i.replace(' ','_')+'_sentences']))

Gene
93627
RNA
2327
Protein
2293
Carbohydrate
18257
Lipid
17200
Peptide
3809
Pharmaceutical_Preparations
292
Toxin
578
Mutation
4130
Disease
750413


# Entity_Info.json

In [7]:
all_es_result_dict={}
for i in all_es_result:
    if not all_es_result_dict.get(i.get('entity')):
        all_es_result_dict[i.get('entity')]=i

In [8]:
all_es_result[10000]

{'entity': 'Fractures, Malunited',
 'PMID': ['34753888',
  '24621174',
  '22357080',
  '36115618',
  '19723984',
  '30827706'],
 'target': [['humerus varus malunion'],
  ['fracture malunion'],
  ['fracture malunions', 'fracture malunions'],
  ['fracture malunion'],
  ['painless malunion'],
  ['fracture malunions']],
 'date': [20220601, 20140312, 20120701, 20221201, 20090901, 20190701],
 'type': 'Disease',
 'sentence': [['Humeral stem prosthesis implantation in case of proximal humerus varus malunion (type 1D fracture sequelae) is often complicated by greater tuberosity fracture and by posterosuperior rotator cuff iatrogenic damage.'],
  ['Both techniques are associated with complications but the limited published research indicates higher complication rates of fracture malunion (poor position at healing) with casting.'],
  ['In the LP group, three (9%) sustained fracture nonunions, three (9%) sustained fracture malunions, and two (6%) sustained surgical site infections.',
   'In the RI

In [9]:
all_es_dict={}
for i in all_es:
    if not i.get('url'):
        i['url']=i.get('NCBI_URL')
    j={'entity':i.get('entity'),'type':i.get('type'),'PMID':i.get('PMID'),
       'official full name':i.get('Official_Full_Name'),
       'sentence':all_es_result_dict.get(i.get('entity')).get('sentence'),
      'numbers of articles':i.get('literature'),
       'JT':all_es_result_dict.get(i.get('entity')).get('JT'),
       'TA':all_es_result_dict.get(i.get('entity')).get('TA'),
       'IF':all_es_result_dict.get(i.get('entity')).get('IF'),
       'IF5':all_es_result_dict.get(i.get('entity')).get('IF5'),
       'year':all_es_result_dict.get(i.get('entity')).get('year'),
       'date':all_es_result_dict.get(i.get('entity')).get('date'),
       'alias names':i.get('Also_known_as',''),
       'description':i.get('Summary',''),
       'url':i.get('url',''),
       'mutation position':i.get('position',''),
       'mutation alleles':i.get('Alleles',''),
       'MeSH ID':i.get('MeshID',''),
       'relation':i.get('network'),
       'external links':[],
       'aging biomarker':i.get('aging'),
       'longevity biomarker':i.get('longevity')}
    if i.get('NCBI_URL'):
        j['external links']+=[{'NCBI':{'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID')}}]
    if i.get('GenAge_URL'):
        j['external links']+=[{'GenAge':{'GenAge_URL':i.get('GenAge_URL'),'GenAge_symbol':i.get('GenAge_symbol')}}]
    if i.get('LongevityMap_URL'):
        j['external links']+=[{'LongevityMap':{'LongevityMap_URL':i.get('LongevityMap_URL'),'LongevityMap_symbol':i.get('LongevityMap_symbol')}}]
    if i.get('AgingBank_URL'):
        j['external links']+=[{'AgingBank':{'AgingBank_URL':i.get('AgingBank_URL'),'AgingBank_symbol':i.get('AgingBank_symbol')}}]
    if i.get('AgingAtlas_URL'):
        j['external links']+=[{'AgingAtlas':{'AgingAtlas_URL':i.get('AgingAtlas_URL'),'AgingAtlas_symbol':i.get('AgingAtlas_symbol')}}]
    if not all_es_dict.get(i.get('entity')):
        all_es_dict[i.get('entity')]=[j]
    else:
        all_es_dict[i.get('entity')]=all_es_dict.get(i.get('entity'))+[j]

In [10]:
len(all_es_dict)

12227

In [11]:
all_es_length=set()
for i,j in all_es_dict.items():
    for k in j[0].get('PMID'):
        all_es_length.add(k)
print(len(all_es_length))

181924


In [12]:
json.dump(all_es_dict,open('results/step5/Entity_Info.json','w'))

In [13]:
list(all_es_dict.items())[0]

('MLH1',
 [{'entity': 'MLH1',
   'type': 'Gene',
   'PMID': ['12612901',
    '30275527',
    '25311944',
    '22936446',
    '19949675',
    '22406557',
    '23240038',
    '11325821',
    '21042749',
    '25556597',
    '17556535',
    '29425284',
    '22740444',
    '37380216',
    '10954253'],
   'official full name': 'mutL homolog 1',
   'sentence': [['Most such cancers have the CpG island methylator phenotype (CIMP+) with methylation and transcriptional silencing of the mismatch repair gene MLH1.'],
    ['Our group recently demonstrated that aging human HSCs accumulate microsatellite instability coincident with loss of MLH1, a DNA Mismatch Repair (MMR) protein, which could reasonably predispose to radiation-induced HSC malignancies.',
     'In addition, whole-exome sequencing analysis revealed high SNVs and INDELs in lymphomas being driven by loss of Mlh1 and frequently mutated genes had a strong correlation with human leukemias.'],
    ['ARID1A loss was observed in 9% (22/257) of

In [551]:
cs=0
for i,j in all_es_dict.items():
    for l in j:
        if k.get('external links'):
            for l in k.get('external links'):
                if l.get('AgingBank'):
                    print(l.get('AgingBank'))
                    cs+=1
    if cs!=0:
        break

{'AgingBank_URL': None, 'AgingBank_symbol': None}


In [ ]:
k

# Relation_Info.json

In [302]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [303]:
Relation_Info_list=[]
Relation_Info_dict={}
for i in triplelistnew:
    if i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed') not in Relation_Info_dict.keys():
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]={
            'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':[i.get('sentence')],
            'source':[i.get('entity_one')],
            'target':[i.get('entity_two')],
            'source type':entitydict_tolist.get(i.get('entity_one_transformed')),
            'target type':entitydict_tolist.get(i.get('entity_two_transformed')),
            'PMID':[i.get('PMID')],
            'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
            'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
            'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
            'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
            'IF':[i.get('IF')],
            'IF5':[i.get('IF5')],
            'method':i.get('method')}
    else:
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['PMID']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('PMID')+[i.get('PMID')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['sentence']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('sentence')+[i.get('sentence')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['source']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('source')+[i.get('entity_one')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['target']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('target')+[i.get('entity_two')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['DP']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['TI']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['TA']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['date']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['IF']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('IF')+[i.get('IF')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['IF5']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('IF5')+[i.get('IF5')]
        if i.get('method')[0] not in Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('method'):
            Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['method']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('method')+i.get('method')
        else:
            continue
            

In [304]:
list(Relation_Info_dict.items())[10]

('Sarcopenia-associated-Constipation',
 {'source entity': 'Sarcopenia',
  'relationship': 'associated',
  'target entity': 'Constipation',
  'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
  'source': ['Sarcopenia'],
  'target': ['constipation'],
  'source type': ['Disease'],
  'target type': ['Disease'],
  'PMID': ['34769606'],
  'DP': ['2021 Oct 21'],
  'date': [20211021],
  'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
  'TA': ['Int J Environ Res Public Health'],
  'IF': [4.614],
  'IF5': [4.798],
  'method': ['open domain']})

In [305]:
len(Relation_Info_dict)

115522

In [306]:
Relation_Info_length=set()
for i,j in Relation_Info_dict.items():
    for k in j.get('PMID'):
        Relation_Info_length.add(k)
print(len(Relation_Info_length))

50191


In [307]:
json.dump(Relation_Info_dict,open('results/step5/Relation_Info.json','w'))

# Aging_Biomarkers.json

In [308]:
len(positive_relations)

8330

In [309]:
positive_relations[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [310]:
pubtator_info_dict_TI={}
pubtator_info_dict_DP={}
for i in pubtator_info:
    pubtator_info_dict_TI[i.get('PMID')]=i.get('TI')
for i in pubtator_info:
    pubtator_info_dict_DP[i.get('PMID')]=i.get('DP')

In [311]:
Aging_Biomarkers_dict={}
for i in positive_relations:
    if not Aging_Biomarkers_dict.get('entity_one_transformed'):
        Aging_Biomarkers_dict[i.get('entity_one_transformed')]=[{
        'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':i.get('sentence'),
            'source':i.get('entity_one'),
            'target':i.get('entity_two'),
            'source type':i.get('entity_one_type'),
            'target type':i.get('entity_two_type'),
            'PMID':i.get('PMID'),
            'DP':pubtator_info_dict_DP.get(i.get('PMID')),
            'date':i.get('date'),
            'TI':pubtator_info_dict_TI.get(i.get('PMID')),
            'TA':i.get('TA'),
            'IF':i.get('IF'),
            'IF5':i.get('IF5')
        }]

In [312]:
list(Aging_Biomarkers_dict.items())[0]

('GPT',
 [{'source entity': 'GPT',
   'relationship': 'correlated',
   'target entity': 'Death',
   'sentence': 'RESULTS: Profiling of blood parameters demonstrated that elevated levels of alanine aminotransferase (ALT), total bilirubin (T-bil), blood urea nitrogen (BUN), creatinine (Cr) and a decreased platelet count were significantly correlated with death within 1 week in a training cohort.',
   'source': 'alanine aminotransferase',
   'target': 'death',
   'source type': 'Gene',
   'target type': 'Disease',
   'PMID': '28011502',
   'DP': '2017 Jan',
   'date': 20170101,
   'TI': 'Objective Predictive Score as a Feasible Biomarker for Short-term Survival in TerminalIy Ill Patients with Cancer.',
   'TA': 'Anticancer Res',
   'IF': 2.435,
   'IF5': 2.376}])

In [313]:
len(Aging_Biomarkers_dict)

1855

In [314]:
Aging_Biomarkers_length=set()
for i,j in Aging_Biomarkers_dict.items():
    for k in j:
        Aging_Biomarkers_length.add(k.get('PMID'))
print(len(Aging_Biomarkers_length))

1502


In [315]:
json.dump(Aging_Biomarkers_dict,open('results/step5/Aging_Biomarkers.json','w'))

# Longevity_Biomarkers.json

In [316]:
Longevity_Biomarkers_dict={}
for i in negative_relations:
    if not Longevity_Biomarkers_dict.get('entity_one_transformed'):
        Longevity_Biomarkers_dict[i.get('entity_one_transformed')]=[{
        'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':i.get('sentence'),
            'source':i.get('entity_one'),
            'target':i.get('entity_two'),
            'source type':i.get('entity_one_type'),
            'target type':i.get('entity_two_type'),
            'PMID':i.get('PMID'),
            'DP':pubtator_info_dict_DP.get(i.get('PMID')),
            'date':i.get('date'),
            'TI':pubtator_info_dict_TI.get(i.get('PMID')),
            'TA':i.get('TA'),
            'IF':i.get('IF'),
            'IF5':i.get('IF5')
        }]

In [317]:
list(Longevity_Biomarkers_dict.items())[0]

('Glucose',
 [{'source entity': 'Glucose',
   'relationship': 'prevent',
   'target entity': 'Diabetes Mellitus',
   'sentence': 'There are data to suggest that control of blood glucose levels will prevent damaging glucotoxicity, preventing or delaying diabetes-related complications.',
   'source': 'glucose',
   'target': 'diabetes',
   'source type': 'Carbohydrate',
   'target type': 'Disease',
   'PMID': '11912813',
   'DP': '2000 Nov-Dec',
   'date': 20001101,
   'TI': 'Elevated plasma glucose levels increase risk for complications.',
   'TA': 'Diabetes Educ',
   'IF': 0.0,
   'IF5': 0.0}])

In [318]:
len(Longevity_Biomarkers_dict)

525

In [319]:
Longevity_Biomarkers_length=set()
for i,j in Longevity_Biomarkers_dict.items():
    for k in j:
        Longevity_Biomarkers_length.add(k.get('PMID'))
print(len(Longevity_Biomarkers_length))

494


In [320]:
json.dump(Longevity_Biomarkers_dict,open('results/step5/Longevity_Biomarkers.json','w'))

## Evaluation

In [347]:
num_of_aging=set()
num_of_longevity=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        num_of_aging.add(i.get('entity_one_transformed'))
    elif i.get('partofspeech')=='negative':
        num_of_longevity.add(i.get('entity_one_transformed'))
print(len(num_of_aging))
print(len(num_of_longevity))

1855
525


In [348]:
all_es_count=set()
print(len(all_es))
for i in all_es:
    all_es_count.add(i.get('entity'))
print(len(all_es_count))

12283
12227


In [349]:
print(len(relation_interpolation_list))

115522


In [350]:
disease_entity_list=[]
disease_entity_triple=set()
disease_entity_dict={}
for i in disease_list:
    if i.get('partofspeech')=='positive':
        if i.get('entity_one_type')=='Gene':
            # if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')) not in disease_entity_triple:

            disease_entity_triple.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list.append(i)
            disease_entity_dict[i.get('entity_one_transformed')]=disease_entity_dict.get(i.get('entity_one_transformed'),0)+1

In [351]:
len(disease_entity_dict)

1353

In [352]:
disease_entity_dict_sorted = sorted(disease_entity_dict.items(), key=lambda x: x[1], reverse=True)

In [353]:
len(disease_entity_dict_sorted)

1353

In [364]:
for i in disease_entity_dict_sorted[:10]:
    print(i)

('INS', 474)
('CRP', 239)
('APOE', 180)
('KL', 175)
('MAPT', 163)
('SIRT1', 103)
('IL6', 88)
('ADIPOQ', 80)
('CD4', 74)
('LEP', 70)


In [361]:
for i in disease_entity_dict_sorted[:10]:
    print(i[0])

INS
CRP
APOE
KL
MAPT
SIRT1
IL6
ADIPOQ
CD4
LEP


In [356]:
disease_entity_list2=[]
disease_entity_triple2=set()
disease_entity_dict2={}
for i in disease_list:
    if i.get('partofspeech')=='negative':
        if i.get('entity_one_type')=='Gene':
            # if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')) not in disease_entity_triple:

            disease_entity_triple2.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list2.append(i)
            disease_entity_dict2[i.get('entity_one_transformed')]=disease_entity_dict2.get(i.get('entity_one_transformed'),0)+1

In [357]:
disease_entity_dict2_sorted = sorted(disease_entity_dict2.items(), key=lambda x: x[1], reverse=True)

In [358]:
len(disease_entity_dict2_sorted)

396

In [362]:
for i in disease_entity_dict2_sorted[:10]:
    print(i)

('INS', 66)
('SIRT1', 63)
('KL', 52)
('NFE2L2', 28)
('ACE', 22)
('MAPT', 20)
('SIRT6', 20)
('TP53', 19)
('APOE', 16)
('PRKAA2', 15)


In [363]:
for i in disease_entity_dict2_sorted[:10]:
    print(i[0])

INS
SIRT1
KL
NFE2L2
ACE
MAPT
SIRT6
TP53
APOE
PRKAA2


## GenAge

In [365]:
Genedict=json.load(open('../dict/Genedict.json'))

In [366]:
Genedict_upper={}
for i,j in Genedict.items():
    temp_list=[]
    for k in j:
        if k not in temp_list:
            temp_list.append(k)
    Genedict_upper[i.upper()]=temp_list

In [367]:
import os
import pandas as pd

import csv
import pickle
import json

In [368]:
genage=pd.read_csv('../other_databases/genage_human.csv')

In [369]:
genage[:2]

,GenAge ID,symbol,name,entrez gene id,uniprot,why
0,1,GHR,growth hormone receptor,2690,GHR_HUMAN,mammal
1,2,GHRH,growth hormone releasing hormone,2691,SLIB_HUMAN,mammal


In [370]:
len(genage)

307

In [372]:
# already check that one to one
genage_symbol=set()
genage_symbol_transfer={}
for i in genage['symbol'].tolist():
    if Genedict_upper.get(i.upper()):
        genage_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
    else:
        # genage_symbol_transfer[i]=i
        print(i)

TERC


In [374]:
len(genage_symbol_transfer)

306

In [377]:
genage_hald_aging=0
for i in disease_entity_dict_sorted:
    if genage_symbol_transfer.get(i[0].upper()):
        genage_hald_aging+=1
genage_hald_aging

166

In [380]:
genage_hald_longevity=0
for i in disease_entity_dict2_sorted:
    if genage_symbol_transfer.get(i[0].upper()):
        genage_hald_longevity+=1
genage_hald_longevity

82

## LongevityMap

In [382]:
longevity_human=pd.read_csv('../other_databases/longevity_human.csv')

In [383]:
len(longevity_human)

550

In [384]:
longevity_human_genes_symbol=set()
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            longevity_human_genes_symbol.add(i)
print(len(longevity_human_genes_symbol))

884


In [385]:
longevity_human_symbol=set()
longevity_human_symbol_transfer={}
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            if Genedict_upper.get(i.upper()) and len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
                print(i)
                print(Genedict_upper.get(i.upper()))
                break

In [386]:
# already check that one to one
longevity_human_symbol=set()
longevity_human_symbol_transfer={}
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            if Genedict_upper.get(i.upper()):
                longevity_human_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i


In [387]:
len(longevity_human_symbol_transfer)

766

In [388]:
longevitymap_hald_aging=0
for i in disease_entity_dict_sorted:
    if longevity_human_symbol_transfer.get(i[0].upper()):
        longevitymap_hald_aging+=1
longevitymap_hald_aging

225

In [389]:
longevitymap_hald_longevity=0
for i in disease_entity_dict2_sorted:
    if longevity_human_symbol_transfer.get(i[0].upper()):
        longevitymap_hald_longevity+=1
longevitymap_hald_longevity

80

## AgingBank

In [390]:
agingbank_genes_pro=pd.read_csv('../other_databases/agingbank_genes_pro.csv')

In [391]:
agingbank_genes_pro[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,ABI3,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
1,Homo sapiens,gene,ABI3BP,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
2,Homo sapiens,gene,ACE,human,NaN,NaN,Pro,16971231,details


In [392]:
agingbank_genes_pro_list=agingbank_genes_pro['name'].tolist()

In [393]:
len(agingbank_genes_pro_list)

97

In [394]:
# already check that one to one except p21 possibly for ['CDKN1A', 'TCEAL1']
# already check that no RNA
agingbank_genes_pro_symbol=set()
agingbank_genes_pro_symbol_transfer={}
for i in agingbank_genes_pro_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[1]]=i
        else:
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i

In [395]:
len(agingbank_genes_pro_symbol_transfer)

55

In [404]:
agingbank_hald_aging=0
for i in disease_entity_dict_sorted:
    if agingbank_genes_pro_symbol_transfer.get(i[0].upper()):
        agingbank_hald_aging+=1
agingbank_hald_aging

32

In [405]:
agingbank_hald_longevity=0
for i in disease_entity_dict2_sorted:
    if agingbank_genes_pro_symbol_transfer.get(i[0].upper()):
        agingbank_hald_longevity+=1
agingbank_hald_longevity

19

In [417]:
agingbank_genes_anti=pd.read_csv('../other_databases/agingbank_genes_anti.csv')

In [418]:
agingbank_genes_anti[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,AEG-1,cell line,LNCaP;C42B,Down,Anti,28923415,details
1,Homo sapiens,gene,AKT1,human,Brain,NaN,Anti,16914869,details
2,Homo sapiens,gene,APOE,human,NaN,NaN,Anti,17234815,details


In [419]:
agingbank_genes_anti_list=agingbank_genes_anti['name'].tolist()

In [420]:
len(agingbank_genes_anti_list)

154

In [421]:
# already check that one to one except p21 possibly for ['CDKN1A', 'TCEAL1']
# already check that no RNA
agingbank_genes_anti_symbol=set()
agingbank_genes_anti_symbol_transfer={}
for i in agingbank_genes_anti_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[1]]=i
        else:
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i

In [422]:
len(agingbank_genes_anti_symbol_transfer)

99

In [423]:
agingbank_hald_aging=0
for i in disease_entity_dict_sorted:
    if agingbank_genes_anti_symbol_transfer.get(i[0].upper()):
        agingbank_hald_aging+=1
agingbank_hald_aging

51

In [424]:
agingbank_hald_longevity=0
for i in disease_entity_dict2_sorted:
    if agingbank_genes_anti_symbol_transfer.get(i[0].upper()):
        agingbank_hald_longevity+=1
agingbank_hald_longevity

27

In [425]:
agingbank_genes_anti_symbol_transfer.get('MLH1')

'MLH1'

## AgingAtlas

In [397]:
agingatlas_genes=pd.read_csv('../other_databases/agingatlas_genes.csv')

In [398]:
agingatlas_genes[:3]

,Symbol
0,IL2RB
1,PSAT1
2,A2M


In [399]:
agingatlas_genes_list=agingatlas_genes['Symbol'].tolist()

In [400]:
len(agingatlas_genes_list)

502

In [402]:
# already check that one to one
# already check that RNA only TERC
agingatlas_genes_symbol=set()
agingatlas_genes_symbol_transfer={}
for i in agingatlas_genes_list:
    if Genedict_upper.get(i.upper()):
        agingatlas_genes_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i


In [406]:
agingatlas_hald_aging=0
for i in disease_entity_dict_sorted:
    if agingatlas_genes_symbol_transfer.get(i[0].upper()):
        agingatlas_hald_aging+=1
agingatlas_hald_aging

238

In [407]:
agingatlas_hald_longevity=0
for i in disease_entity_dict2_sorted:
    if agingatlas_genes_symbol_transfer.get(i[0].upper()):
        agingatlas_hald_longevity+=1
agingatlas_hald_longevity

110

In [446]:
x=[
'disease_entity_dict',
'disease_entity_dict2',
'genage_symbol_transfer',
'longevity_human_symbol_transfer',
'agingbank_genes_pro_symbol_transfer',
'agingbank_genes_anti_symbol_transfer',
'agingatlas_genes_symbol_transfer',
]

In [451]:
names=locals()
for i in x:
    print(i)
    print(len(names[i]))

disease_entity_dict
1353
disease_entity_dict2
396
genage_symbol_transfer
306
longevity_human_symbol_transfer
766
agingbank_genes_pro_symbol_transfer
55
agingbank_genes_anti_symbol_transfer
99
agingatlas_genes_symbol_transfer
500


In [428]:
print(len(disease_entity_dict_sorted))
print(len(disease_entity_dict2_sorted))

1353
396


In [415]:
ccc=0
for i in genage_symbol_transfer.keys():
    if i in agingatlas_genes_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in genage_symbol_transfer.keys():
    if i in agingbank_genes_pro_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in agingatlas_genes_symbol_transfer.keys():
    if i in agingbank_genes_pro_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol_transfer.keys() and i[0] in agingatlas_genes_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol_transfer.keys() and i[0] in agingbank_genes_pro_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in agingatlas_genes_symbol_transfer.keys() and i[0] in agingbank_genes_pro_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in genage_symbol_transfer.keys():
    if i in agingatlas_genes_symbol_transfer.keys() and i in agingbank_genes_pro_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol_transfer.keys() and i[0] in agingatlas_genes_symbol_transfer.keys() and i[0] in agingbank_genes_pro_symbol_transfer.keys() :
        ccc+=1
print(ccc)

306
16
20
166
14
17
16
14


In [426]:
ccc=0
for i in longevity_human_symbol_transfer.keys():
    if i in agingbank_genes_anti_symbol_transfer.keys():
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict2_sorted:
    if i[0] in longevity_human_symbol_transfer.keys() and i[0] in agingbank_genes_anti_symbol_transfer.keys():
        ccc+=1
print(ccc)

33
21


In [470]:
list(disease_entity_dict.items())[0]

('GPT', 13)

In [485]:
disease_entity_dict_all={}
for i in disease_entity_dict.keys():
    disease_entity_dict_all[i]=disease_entity_dict_all.get(i,0)+disease_entity_dict.get(i)
for i in disease_entity_dict2.keys():
    disease_entity_dict_all[i]=disease_entity_dict_all.get(i,0)+disease_entity_dict2.get(i)
len(disease_entity_dict_all)

1473

In [494]:
disease_entity_dict_all_sorted = sorted(disease_entity_dict_all.items(), key=lambda x: x[1], reverse=True)

In [473]:
agingbank_all={}
for i in agingbank_genes_pro_symbol_transfer.keys():
    agingbank_all[i]=i
for i in agingbank_genes_anti_symbol_transfer.keys():
    agingbank_all[i]=i
len(agingbank_all)

144

In [475]:
y=[
'genage_symbol_transfer',
'longevity_human_symbol_transfer',
'agingbank_all',
'agingatlas_genes_symbol_transfer',
]

In [505]:
for i in y:
    common=0
    count=0
    for j in disease_entity_dict_all.keys():
        if names[i].get(j):
            count+=1
    for j in disease_entity_dict_all_sorted[:10]:
        if names[i].get(j[0]):
            common+=1
    print(i+'  '+str(count)+'  '+str(count/len(names[i]))+'  '+str(common))

genage_symbol_transfer  173  0.565359477124183  7
longevity_human_symbol_transfer  235  0.30678851174934724  7
agingbank_all  77  0.5347222222222222  7
agingatlas_genes_symbol_transfer  247  0.494  8


In [481]:
z1=[
'genage_symbol_transfer',
'agingbank_genes_pro_symbol_transfer',
'agingatlas_genes_symbol_transfer',
]

In [506]:
for i in z1:
    common=0
    count=0
    for j in disease_entity_dict.keys():
        if names[i].get(j):
            count+=1
    for j in disease_entity_dict_sorted[:10]:
        if names[i].get(j[0]):
            common+=1
    print(i+'  '+str(count)+'  '+str(count/len(names[i]))+'  '+str(common))

genage_symbol_transfer  166  0.5424836601307189  7
agingbank_genes_pro_symbol_transfer  32  0.5818181818181818  3
agingatlas_genes_symbol_transfer  238  0.476  8


In [507]:
z2=[
'longevity_human_symbol_transfer',
'agingbank_genes_anti_symbol_transfer',
]

In [508]:
for i in z2:
    common=0
    count=0
    for j in disease_entity_dict2.keys():
        if names[i].get(j):
            count+=1
    for j in disease_entity_dict2_sorted[:10]:
        if names[i].get(j[0]):
            common+=1
    print(i+'  '+str(count)+'  '+str(count/len(names[i]))+'  '+str(common))

longevity_human_symbol_transfer  80  0.10443864229765012  7
agingbank_genes_anti_symbol_transfer  27  0.2727272727272727  7


In [479]:
x=[
'disease_entity_dict',
'disease_entity_dict2',
'genage_symbol_transfer',
'longevity_human_symbol_transfer',
'agingbank_genes_pro_symbol_transfer',
'agingbank_genes_anti_symbol_transfer',
'agingatlas_genes_symbol_transfer',
]

In [453]:
HALD_genes_transfer={}
aging_genes_set=set()
for i in disease_entity_dict_sorted:
    HALD_genes_transfer[i[0]]=i[0]
    aging_genes_set.add(i[0])

In [454]:
len(list(aging_genes_set))

1353

In [455]:
print(len(genage_symbol_transfer))
for i in genage_symbol_transfer.keys():
    aging_genes_set.add(i)
print(len(aging_genes_set))
print(len(agingbank_genes_pro_symbol_transfer))
for i in agingbank_genes_pro_symbol_transfer.keys():
    aging_genes_set.add(i)
print(len(aging_genes_set))
print(len(agingatlas_genes_symbol_transfer))
for i in agingatlas_genes_symbol_transfer.keys():
    aging_genes_set.add(i)
print(len(aging_genes_set))

306
1493
55
1514
500
1635


In [456]:
aging_genes_list=list(aging_genes_set)

In [458]:
HALD_01=[]
genage_symbol_01=[]
agingbank_genes_pro_symbol_01=[]
agingatlas_genes_symbol_01=[]
for i in aging_genes_list:
    if HALD_genes_transfer.get(i):
        HALD_01.append(1)
    else:
        HALD_01.append(0)
    if genage_symbol_transfer.get(i):
        genage_symbol_01.append(1)
    else:
        genage_symbol_01.append(0)
    if agingbank_genes_pro_symbol_transfer.get(i):
        agingbank_genes_pro_symbol_01.append(1)
    else:
        agingbank_genes_pro_symbol_01.append(0)
    if agingatlas_genes_symbol_transfer.get(i):
        agingatlas_genes_symbol_01.append(1)
    else:
        agingatlas_genes_symbol_01.append(0)

In [459]:
vfdsvg=0
for i in HALD_genes_transfer.keys():
    if not genage_symbol_transfer.get(i) and  agingbank_genes_pro_symbol_transfer.get(i) and  agingatlas_genes_symbol_transfer.get(i):
        vfdsvg+=1
vfdsvg

3

In [460]:
dataframe5 = pd.DataFrame({'HALD':HALD_01,'GenAge':genage_symbol_01,'Aging Atlas':agingatlas_genes_symbol_01,'AgingBank (Pro)':agingbank_genes_pro_symbol_01})
dataframe5.to_csv("results/step4/upset1.csv", index=False,sep=',')

In [461]:
HALD_longevity_genes_transfer={}
longevity_genes_set=set()
for i in disease_entity_dict2_sorted:
    HALD_longevity_genes_transfer[i[0]]=i[0]
    longevity_genes_set.add(i[0])

In [462]:
len(list(longevity_genes_set))

396

In [463]:
print(len(longevity_human_symbol_transfer))
for i in longevity_human_symbol_transfer.keys():
    longevity_genes_set.add(i)
print(len(longevity_genes_set))
print(len(agingbank_genes_anti_symbol_transfer))
for i in agingbank_genes_anti_symbol_transfer.keys():
    longevity_genes_set.add(i)
print(len(longevity_genes_set))


766
1082
99
1142


In [464]:
longevity_genes_list=list(longevity_genes_set)

In [465]:
HALD_longevity_01=[]
longevity_human_symbol_01=[]
agingbank_genes_anti_symbol_01=[]

for i in longevity_genes_list:
    if HALD_longevity_genes_transfer.get(i):
        HALD_longevity_01.append(1)
    else:
        HALD_longevity_01.append(0)
    if longevity_human_symbol_transfer.get(i):
        longevity_human_symbol_01.append(1)
    else:
        longevity_human_symbol_01.append(0)
    if agingbank_genes_anti_symbol_transfer.get(i):
        agingbank_genes_anti_symbol_01.append(1)
    else:
        agingbank_genes_anti_symbol_01.append(0)

In [466]:
dataframe6 = pd.DataFrame({'HALD':HALD_longevity_01,'LongevityMap':longevity_human_symbol_01,'AgingBank (Anti)':agingbank_genes_anti_symbol_01})
dataframe6.to_csv("results/step4/upset2.csv", index=False,sep=',')